In [9]:
# Imports
import curses
import time
import sys
import math
import os
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as img
import csv
import numpy as np
from collections import deque
from random import randint
from PIL import Image

# Tensorflow imports
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D,Conv1D, MaxPooling2D,MaxPooling1D, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber
import tensorflow.python.keras.backend as backend



In [10]:
# RL settings
DISCOUNT = 0.89
REPLAY_MEMORY_SIZE = 10000  
MIN_REPLAY_MEMORY_SIZE = 1000 
MINIBATCH_SIZE = 128  
UPDATE_TARGET_EVERY = 5  
MODEL_NAME = 'snake'
LEARNING_RATE = 0.005

# Environment settings
EPISODES = 2000
ENV_SIZE = 10

# Exploration settings
EPSILON_DECAY = 0.95
MIN_EPSILON = 0.05

# Stats settings
AGGREGATE_STATS_EVERY = 25  
SHOW_PREVIEW = False

In [11]:
class Field:
    #Initialize game field
    def __init__(self,size):
        self.size = size
        # Used for terminal print
        self.icons = {
            0: ' . ',
            1: ' * ',
            2: ' # ',
            3: ' & ',
            4 : ' 0 '
        }
        # Used for RGB image
        self.d = {
            0: (255, 255, 255),
            1: (216, 173, 230),
            2: (0, 0, 255),
            3: (0,255,0),
            4: (0,0,0)
            }

        self.snake_coords = [[5, 3], [5, 4], [5, 5]]
        self._generate_field()
        self.add_entity()
        if self.get_entity_pos()==[-1,-1]:
            self.add_entity()

    def reset(self):
        # Reset field after episode
        self._generate_field()
        self.add_entity()
        self.snake_coords = [[5, 3], [5, 4], [5, 5]]
        if self.get_entity_pos()==[-1,-1]:
            self.add_entity()
        
    def add_entity(self):  
       # Add food to field 
        while(True):
            i = randint(2, self.size-2)
            j = randint(2, self.size-2)
            entity = [i, j]
            
            if entity not in self.snake_coords:
                self.field[i][j] = 3
                break
 
    def _generate_field(self):
        self.field = np.zeros((self.size, self.size), dtype=np.uint8) 
    
    def _clear_field(self):        
        self.field = [[j if j!= 1 and j!= 2 else 0 for j in i] for i in self.field]

    def render(self, show):
        # Render field enumeration 
        size = self.size
        self._clear_field()

        # Render snake on the field
        for i, j in self.snake_coords:
            self.field[i][j] = 1
        if self.get_entity_pos()==[-1,-1]:
            self.add_entity()

        # Mark head
        head = self.snake_coords[len(self.snake_coords)-1]
        self.field[head[0]][head[1]] = 2

        for i in range(self.size):
                self.field[i][0] = 4
                self.field[0][i] = 4
                self.field[self.size-1][i] = 4
                self.field[i][self.size-1] = 4

        # Print icons to terminal
        if show:
            for i in range(self.size):
                row = ''
                for j in range(self.size):
                    row += self.icons[ self.field[i][j] ]
                print(row,'\n')   
        return np.array(self.field)


    def get_image(self,game_field):
        # Convert enumerated field to RGB image
        env = np.zeros((self.size, self.size, 3), dtype=np.uint8) 
        
        for i in range(self.size):
            for j in range(self.size):
                if game_field[i][j]==0:
                    env[i][j] = self.d[0]
            if game_field[i][j]==1:
                    env[i][j] = self.d[1]
            if game_field[i][j]==2:
                    env[i][j] = self.d[2]
            if game_field[i][j]==3:
                    env[i][j] = self.d[3]
        img = Image.fromarray(env, 'RGB') 
        return img



    def get_entity_pos(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.field[i][j] == 3:
                    return [i, j]
        return [-1, -1]


    def is_snake_eat_entity(self):
        entity = self.get_entity_pos()
        head = self.snake_coords[-1]
        return entity == head


In [12]:
class Snake:
    def __init__(self):
        self.direction = 0
        self.dist =8
        # Init coords
        self.coords = [[5, 3], [5, 4], [5, 5]]

    def reset(self):
        self.direction = 0

        # Init coords
        self.coords = [[5, 3], [5, 4], [5, 5]]
        
    def set_direction_normal(self, ch):

        # Check if wrong direction
        if ch == 1 and self.direction == 0:
            return
        if ch == 0 and self.direction == 1:
            return
        if ch == 3 and self.direction == 2:
            return
        if ch == 2 and self.direction == 3:
            return 

        self.direction = ch

    def set_direction_random(self):
        rand_dir = randint(0, 4)
        # Check if wrong direction
        if rand_dir == 0 and  self.direction != 0:
            self.direction = 1
        if rand_dir == 1 and  self.direction != 1:
            self.direction = 0
        if rand_dir == 2 and self.direction != 2:
            self.direction = 2
        if rand_dir == 3 and self.direction != 3:
            self.direction = 3



    def level_up(self):
        # get last point direction
        a = self.coords[0]
        b = self.coords[1]

        tail = a[:]

        if a[0] < b[0]:
            tail[0]-=1
        elif a[1] < b[1]:
            tail[1]-=1
        elif a[0] > b[0]:
            tail[0]+=1
        elif a[1] > b[1]:
            tail[1]+=1

        tail = self._check_limit(tail)
        self.coords.insert(0, tail)

    def is_dead(self):
        head = self.coords[-1]
        snake_body = self.coords[:-1]

        return head in snake_body

    def hit_wall(self):
        head = self.coords[-1]
        self.walls = []
        for i in range(self.field.size):
            self.walls.append([i,0])
            self.walls.append([0,i])
            self.walls.append([self.field.size-1,i])
            self.walls.append([i,self.field.size-1])

        return head in self.walls


    def _check_limit(self, point):
        # Check field limit
        if point[0] > self.field.size-1:
            point[0] = 0
        elif point[0] < 0:
            point[0] = self.field.size-1
        elif point[1] < 0:
            point[1] = self.field.size-1
        elif point[1] > self.field.size-1:
            point[1] = 0

        return point

    def move(self):
        # Determine head coords
        head = self.coords[-1][:]
        food =  self.field.get_entity_pos()
        
        done=False
        # Calc new head coords
        if self.direction == 3:
            head[0]-=1
        elif self.direction == 2:
            head[0]+=1
        elif self.direction == 0:
            head[1]+=1
        elif self.direction == 1:
            head[1]-=1
        dist=math.sqrt((head[0]-food[0])**2+(head[1]-food[1])**2)
        if(dist<self.dist):
            reward = -0.005
            self.dist = dist
        else:
            reward = -0.01
            self.dist = dist

        # Check field limit
        head = self._check_limit(head)

        del(self.coords[0])
        self.coords.append(head)
        self.field.snake_coords = self.coords

        if  self.is_dead():
            reward = -5
            done = True

        if self.hit_wall():
            reward = -5
            done =True
        # check if snake eat an entity
        if self.field.is_snake_eat_entity():
            #curses.beep()
            self.level_up()
            self.field.add_entity()
            reward = 1
        return done,reward


    def set_field(self, field):
        self.field = field

    def get_entity_pos(self):
            for i in range(ENV_SIZE):
                for j in range(ENV_SIZE):
                    if self.field[i][j] == 3:
                        return [i, j]

            return [-1, -1]

In [13]:
class Agent:
    def __init__(self):

        # Main model
        self.model = self.create_model()

        # Target network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)
        
        self.target_update_counter = 0

    def create_model(self):

            model = Sequential()

            model.add(Conv1D(32,7,strides=2,activation='relu',padding='same', input_shape=(ENV_SIZE, ENV_SIZE)))  
            model.add(MaxPooling1D(pool_size=(3),padding='same'))
            
            model.add(Conv1D(64,3,activation='relu',padding='same'))  
            model.add(MaxPooling1D(pool_size=(2),padding='same'))

            model.add(Flatten()) 
            model.add(Dense(128,activation='relu'))
            model.add(Dense(128,activation='relu'))
            model.add(Dense(4, activation='linear')) 
            model.compile(loss='log_cosh', optimizer=Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])
            return model


    # Adds step's data to a memory(current observation, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)

    # Trains main network every step during episode
    def train(self, terminal_state, step):

        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return

        # Get a minibatch of random samples 
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        # Get current states from minibatch, and Q values from NN
        current_states = np.array([transition[0] for transition in minibatch])/4
        current_qs_list = self.model.predict(current_states)

        # Get future states from minibatch and Q values from NN
        new_current_states = np.array([transition[3] for transition in minibatch])/4
        future_qs_list = self.target_model.predict(new_current_states)
    
        X = []
        y = []

        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state calc new Q-value
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)

        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X)/4, np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False)

        # Update target counter
        if terminal_state:
            self.target_update_counter += 1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space 
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape)/4)[0]

In [ ]:
def main():
    ep_rewards = []
    ep_score = []
    avgs = []
    max_rewards = []
    epsilon = 0.01
    random.seed(1)
    np.random.seed(1)
    tf.random.set_seed(1)

    if not os.path.isdir('models'):
        os.makedirs('models')
    if not os.path.isdir('images'):
        os.makedirs('images')
        
    field = Field(ENV_SIZE)
    snake = Snake()
    snake.set_field(field)
    agent = Agent()

    for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):
        

        # Restarting episode - reset episode reward and step number
        episode_reward = 0
        score = 0
        step = 1
        new_step = 0
        average_reward = -11
        old_avg = -11

        # Reset environment and get initial state
        field.reset()
        snake.reset()
        current_field = field.render(False)
        current_state = field.get_image(current_field)

        # Reset flag and start iterating until episode ends
        done = False
        action = 0
        while not done:

            if np.random.random() > epsilon:
                # Get action from Q table
                action = np.argmax(agent.get_qs(current_field))
                
            else:
                # Get random action
                action = np.random.randint(0, 4)
          
            snake.set_direction_normal(action)
            done, reward= snake.move()
            if reward == 1:
                score += 1
                new_step = step

            new_field = field.render(False)
            new_state = field.get_image(new_field)

            if (step-new_step) >= 100*(round(episode/10000))+100:
                    new_step = step
                    done = True
                    reward = -5

            episode_reward += reward

            if SHOW_PREVIEW and not episode % AGGREGATE_STATS_EVERY:
                img = current_state.resize((400,400))
                img.save(f'images/training-ep_{episode}step__{step}.png')

            # Every step we update replay memory and train network
            agent.update_replay_memory((current_field, action, reward, new_field, done))
            agent.train(done, step)
            
            current_field = new_field
            current_state = new_state
            step += 1

        # Append episode reward to a list 
        ep_rewards.append(episode_reward)
        ep_score.append(score)
        if not episode % AGGREGATE_STATS_EVERY or episode == 1:
            old_avg = average_reward
            average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
            min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
            max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
            avgs.append(average_reward)
            max_rewards.append(max_reward)
            old_avg = average_reward
            #print(f'{episode}Ep_{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min')
            
            #Save model
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')
                

        # Decay epsilon
        if epsilon > MIN_EPSILON:
            epsilon *= EPSILON_DECAY
            epsilon = max(MIN_EPSILON, epsilon)
        
        with open(f'Max-rewards.csv', 'w', encoding='UTF8') as f:
                        # create the csv writer
            writer = csv.writer(f)

                        # write a row to the csv file
            writer.writerow(max_rewards)

        with open('Score.csv', 'w', encoding='UTF8') as f:
                        # create the csv writer
            writer = csv.writer(f)

                        # write a row to the csv file
            writer.writerow(ep_score)

        with open(f'Average-rewards.csv', 'w', encoding='UTF8') as f:
                        # create the csv writer
            writer = csv.writer(f)

                        # write a row to the csv file
            writer.writerow(avgs)

    

main()

  0%|          | 0/2000 [00:00<?, ?episodes/s]

INFO:tensorflow:Assets written to: models/snake____-5.09max___-5.09avg___-5.09min__1639601531.model/assets


  1%|1         | 24/2000 [00:07<08:25,  3.91episodes/s]

INFO:tensorflow:Assets written to: models/snake____-4.02max___-4.99avg___-5.09min__1639601538.model/assets


  2%|2         | 49/2000 [00:15<08:30,  3.82episodes/s]

...

 47%|####7     | 949/2000 [56:11<57:42,  3.29s/episodes]

INFO:tensorflow:Assets written to: models/snake_____0.51max___-4.30avg___-5.76min__1639604911.model/assets


 49%|####8     | 974/2000 [58:16<1:17:17,  4.52s/episodes]

INFO:tensorflow:Assets written to: models/snake____-3.12max___-4.64avg___-5.43min__1639605032.model/assets


 50%|####9     | 990/2000 [59:45<58:36,  3.48s/episodes]  